In [ ]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Load the CIFAR10 dataset
cifar10_data = CIFAR10(root='data/', train=False, download=True, transform=transforms.ToTensor())
cifer10_classes = cifar10_data.classes

print('データ数:', len(cifar10_data))
print('クラス:', cifer10_classes)

n_images = 25
cifar10_loader = DataLoader(cifar10_data, batch_size=n_images, shuffle=True)
data_iter = iter(cifar10_loader)
images, labels = next(data_iter)

plt.figure(figsize=(10, 10))
for i in range(n_images):
    plt.subplot(5, 5, i+1)
    plt.imshow(np.transpose(images[i], (1, 2, 0)))
    label = cifer10_classes[labels[i]]
    plt.title(label)
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)
plt.show()

In [ ]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([
    transforms.RandomAffine([-30, 30], scale=[0.8, 1.2]),
    transforms.ToTensor()
])

cifar10_data = CIFAR10(root='data/',
                       train=False,
                       download=True,
                       transform=transform)
cifer10_classes = cifar10_data.classes

print('データ数:', len(cifar10_data))
print('クラス:', cifer10_classes)

n_images = 25
cifar10_loader = DataLoader(cifar10_data, batch_size=n_images, shuffle=True)
data_iter = iter(cifar10_loader)
images, labels = next(data_iter)

plt.figure(figsize=(10, 10))
for i in range(n_images):
    plt.subplot(5, 5, i+1)
    plt.imshow(np.transpose(images[i], (1, 2, 0)))
    label = cifer10_classes[labels[i]]
    plt.title(label)
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)
plt.show()

In [ ]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

affine = transforms.RandomAffine([-15, 15], scale=[0.8, 1.2])
flip = transforms.RandomHorizontalFlip(p=0.5)
normalize = transforms.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0))
to_tensor = transforms.ToTensor()

transform_train = transforms.Compose([affine, flip, to_tensor, normalize])
transform_test = transforms.Compose([to_tensor, normalize])
cifar10_train = CIFAR10(root='data/', train=True, download=True, transform=transform_train)
cifar10_test = CIFAR10(root='data/', train=False, download=True, transform=transform_test)

# DataLoaderの設定
batch_size = 64
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(cifar10_test, batch_size=len(cifar10_test), shuffle=False)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # 畳み込み層（入力チャンネル数, フィルタ数, フィルタサイズ）
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2) # プーリング層（領域サイズ, ストライド）
        self.fc1 = nn.Linear(16*5*5, 256) # 全結合層（入力ユニット数, 出力ユニット数）
        self.dropout = nn.Dropout(0.5) # ドロップアウト（ドロップアウト率）
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = Net()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

record_loss_train = []
record_loss_test = []

x_test, t_test = next(iter(test_loader))
x_test, t_test = x_test.to(device), t_test.to(device)

for epoch in range(20):
    net.train()
    loss_train = 0
    for j, (x, t) in enumerate(train_loader):
        x, t = x.to(device), t.to(device)
        y = net(x)
        loss = loss_fn(y, t)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    record_loss_train.append(loss_train)
    
    net.eval()
    y_test = net(x_test)
    loss_test = loss_fn(y_test, t_test).item()
    record_loss_test.append(loss_test)
    
    if epoch%1 == 0:
        print('epoch:', epoch, 'loss_train:', loss_train, 'loss_test:', loss_test)

import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label='Train')
plt.plot(range(len(record_loss_test)), record_loss_test, label='Test')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Error')
plt.show()

In [ ]:
# 正解率
correct = 0
total = 0
net.eval()
for i, (x, t) in enumerate(test_loader):
    x, t = x.to(device), t.to(device)
    y = net(x)
    correct += (y.argmax(1) == t).sum().item()
    total += len(x)
print('正解率:', str(correct/total*100) + '%')

In [ ]:
cifar10_loader = DataLoader(cifar10_test, batch_size=1, shuffle=True)
data_iter = iter(cifar10_loader)
images, labels = next(data_iter)

plt.imshow(np.transpose(images[0], (1, 2, 0)))
plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)
plt.show()

net.eval()
x, t = images.to(device), labels.to(device)
y = net(x)
print('正解:', cifer10_classes[labels[0].item()])
print('予測:', cifer10_classes[y.argmax(1).item()])